In [ ]:
# 필요한 모듈 임포트
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 경로 설정
train_dir = 'car-damage-dataset/data1a/training'  # 학습 데이터 폴더 경로
validation_dir = 'car-damage-dataset/data1a/validation'  # 검증 데이터 폴더 경로

# ImageDataGenerator 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# ImageDataGenerator에서 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),  # 이미지 크기 128x128로 조정
    batch_size=32,
    class_mode='binary'  # 이진 분류
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(256, 256),  # 이미지 크기 128x128로 조정
    batch_size=32,
    class_mode='binary'
)

# 모델 설계
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),  # 은닉층 크기 설정
    Dense(1, activation='sigmoid')  # 이진 분류를 위한 출력층
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=100,  # 에포크 수 설정
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


In [ ]:
# 모델 저장
model.save('car_damage_detection_model.h5')


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# 모델 불러오기
model = tf.keras.models.load_model('car_damage_detection_model.h5')

# 예측할 이미지가 있는 폴더 경로
img_dir = 'car-damage-dataset/test'  # 예측할 이미지 폴더 경로

# 이미지 파일 확장자 목록
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

# 폴더 내 모든 이미지 파일에 대해 반복하며 예측
for img_file in os.listdir(img_dir):
    if img_file.lower().endswith(valid_extensions):  # 이미지 파일만 선택
        img_path = os.path.join(img_dir, img_file)
        try:
            # 이미지 불러오기 및 전처리
            img = image.load_img(img_path, target_size=(256, 256))  # 모델의 입력 크기에 맞게 조정
            img_array = image.img_to_array(img) / 255.0  # 정규화 (0~1 범위)
            img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가 (1, 128, 128, 3)

            # 모델 예측
            prediction = model.predict(img_array)

            predicted_prob = prediction[0][0]
            # 예측 결과 출력 (이진 분류)
            if predicted_prob > 0.5:
                print(f"{img_file}: 비파손 (Damage-free)")
            else:
                print(f"{img_file}: 파손 (Damaged)")

        except Exception as e:
            print(f"Error processing {img_file}: {e}")
